In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
train = pd.read_csv("/home/miroslav/Downloads/dont-overfit-ii/train.csv").drop('id', axis=1)

y_train = train['target']
X_train = train.drop('target', axis=1)


test = pd.read_csv('/home/miroslav/Downloads/dont-overfit-ii/test.csv')
X_test = test.drop('id', axis = 1)

submission = pd.read_csv('/home/miroslav/Downloads/dont-overfit-ii/sample_submission.csv')

In [ ]:
from tpot import TPOTClassifier
from tempfile import mkdtemp

cachedir = mkdtemp()

# pozor potom...vvraj ak ma numericky stlpec menej ako 10 unique hodnot, povazuje ho za categorical!!!
# da sa to ale nastavit v configu, veci co sa tykaju categorical a pod stlpcov:
# https://github.com/EpistasisLab/tpot/issues/549
# https://epistasislab.github.io/tpot/using/#customizing-tpots-operators-and-parameters
model = TPOTClassifier(
     generations=500, # cim viac typ lepsie
#     population_size=500, # cim viac tym lepsie
    verbosity=3, 
    n_jobs=-1, 
    random_state=99,
    #random_state=1,
    scoring='roc_auc',
    use_dask=True,
    cv=10,
    memory=cachedir,
    config_dict='TPOT light', # ked ten parameter vynecham, tak to zlyha
    max_eval_time_mins=30, # maximalny cas na jeden pipeline
    periodic_checkpoint_folder='trash/',
    warm_start=True # vdaka tomuto mozem vypocet kedykolvek prerusit a potom dalsi fit pokracuje kde predosly skoncil
)

In [ ]:
model.fit(X_train.copy(), y_train.copy())

In [ ]:
pd.Series(model.predict_proba(X_test)[:,1]).plot(kind='hist')

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV

from yellowbrick.features import RFECV

viz = RFECV(
    LogisticRegressionCV(
        #cv=10, 
        solver='liblinear',
        class_weight='balanced', penalty='l1'
    ), 
    #cv=10, 
    scoring='roc_auc')

viz.fit(X_train, y_train)
viz.poof()

In [ ]:
import autosklearn.classification

model = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    per_run_time_limit=600,
    #ml_memory_limit=3072,
    n_jobs=8,
    resampling_strategy='cv',
#     tmp_folder='trash/',
#     output_folder='trash/',
    resampling_strategy_arguments={'folds': 10, 'shuffle':True},
)

model.fit(
    X_train.copy(), 
    y_train.copy(), 
    metric=autosklearn.metrics.roc_auc, 
    #X_test=None, y_test=None,
    #feat_type=None,
)
#predictions = cls.predict(X_test)

In [ ]:
model.refit(X_train.copy(), y_train.copy())

In [ ]:
pd.DataFrame(model.cv_results_).sort_values(by='mean_test_score', ascending=False)#.to_csv('/home/miroslav/Desktop/tre.csv')

In [ ]:
X_train[[str(i) for i in range(30)]].hist()

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=150, n_iter=7, random_state=42)
X = svd.fit_transform(X_train)  
print(svd.explained_variance_ratio_.sum())  

In [ ]:
import umap
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

embedding = umap.UMAP().fit_transform(X)
#embedding = TSNE().fit_transform(X_train)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(embedding[:,0], embedding[:,1], c=y_train) #, c=y_train

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=200)

svd.fit(X_train)

svd.explained_variance_ratio_.sum()

In [ ]:
X_train.std()

In [ ]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    #ml_memory_limit=8000,
    n_jobs=8,
    resampling_strategy='cv',
        resampling_strategy_arguments={'folds': 10},
)
cls.fit(X_train.copy(), y_train.copy(), metric=autosklearn.metrics.roc_auc)
#predictions = cls.predict(X_test)

In [ ]:
cls.refit(X_train.copy(), y_train.copy())

In [ ]:
pd.DataFrame(cls.show_models())

In [ ]:
cls.predict_proba(X_test)[:,1]

In [ ]:
pd.DataFrame(cls.cv_results_).sort_values(by='mean_test_score', ascending=False).to_csv('/home/miroslav/Desktop/tre.csv')
#.head(1).T

In [ ]:
cls.predict_proba(X_test)[:,1]

In [ ]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(
    #generations=5, 
    #population_size=20, 
    verbosity=2, 
    n_jobs=-1, 
    scoring='roc_auc',
    use_dask=True,
    cv=10,
    config_dict='TPOT light'
)
tpot.fit(X_train.copy(), y_train.copy())

In [ ]:
list(tpot.pareto_front_fitted_pipelines_)[4]

In [ ]:
tpot.predict_proba(X_test)[:,1].mean()

In [ ]:
tpot.export('tpot_mnist_pipeline.py')

# Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier

param_grid = {'max_depth': [2, 4, 6, 10, 30, 100]}
clf = DecisionTreeClassifier() #64

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(
    max_leaf_nodes=None, min_samples_split=19,
    n_estimators=100, max_depth=None, max_features=0.65133, min_samples_leaf=11, criterion='gini')

param_grid = {
    #'learning_rate': [0.05, 0.1],
    #'n_estimators': [300, 1000],
    #'max_depth': [None],#[2, 3, 10, 30, 50, 100, 150]
}

In [ ]:
from xgboost import XGBClassifier

param_grid = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [300, 1000],
    'max_depth': [2, 3, 10, 30]
}

clf = XGBClassifier() #74

In [ ]:
from sklearn.linear_model import LogisticRegression

# , 10, 100, 1000, , 0.001, 0.01, 0.1, 1, 10, 100, 1000
param_grid = {
    'C': [0.00001, 0.001, 0.01, 0.05, 0.075, 0.09, 0.1, 0.11, 0.15, 0.25, 0.7, 1, 3, 10, 100, 1000],
    'penalty': ['l1', 'l2']
}
clf = LogisticRegression(class_weight='balanced', solver='liblinear') #789

In [ ]:
param_grid = {
    'C': [0.0000000001, 0.000000001, 0.001, 0.01, 0.1, 1, 10], 
    'gamma' : [0.000001, 0.001, 0.01, 0.1, 0.5, 1, 50],
    'kernel':['linear', 'poly', 'rbf', 'sigmoid']
}

from sklearn import svm
clf = svm.SVC(probability=True, class_weight='balanced', kernel='poly') #75

In [ ]:
from sklearn.linear_model import SGDClassifier

param_grid = {
    'l1_ratio': [0, 0.00001, 0.01, 0.5, 0.99, 0.9999, 1], 
    'loss': ['hinge', 'log', 'perceptron'],
    'alpha':[ 0.0001, 0.01, 0.1, 0.5, 1, 100]
}

clf = SGDClassifier(penalty='elasticnet', class_weight='balanced') #781

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# CV

In [ ]:
print(__doc__)

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification


rfecv = RFECV(
    estimator=clf,
    step=1,
    n_jobs=-1, 
    #min_features_to_select=50,
    cv=RepeatedStratifiedKFold(n_splits=10, n_repeats = 4, random_state=42),  
    scoring='roc_auc')
rfecv.fit(X_train, y_train)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.predict_proba(X_test)

In [ ]:
#rfecv.ranking_[:8]

X_train[:, rfecv.ranking_ == 1].shape

In [ ]:
rfecv.

In [ ]:
rfecv.ranking_ < 100

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneOut

from sklearn.model_selection import RepeatedStratifiedKFold
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=42)

model = GridSearchCV(
    clf, param_grid, 
    scoring='roc_auc',
    verbose=0, n_jobs = -1, cv = rskf, iid=False
)
model.fit(X_train, y_train).best_score_

In [ ]:
#clf.fit(X_train, y_train)
sum(rfecv.ranking_ < 100)
clf

In [ ]:
model.best_params_

In [ ]:
submission['target'] = model.predict_proba(X_test)[:,1]
#tpot.predict_proba(X_test)[:,1]
#cls.predict_proba(X_test)[:,1]
#cls.predict_proba(X_test)[:,1]
#model.predict_proba(X_test)[:,1] #[:, rfecv.ranking_ < 100]
submission.to_csv('/home/miroslav/Desktop/submission_tpot_0.9 AUC (CV).csv', index=False)

In [ ]:
model.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import Normalizer

gus = GenericUnivariateSelect(f_classif, 'fpr' ,param=0.1847500865330056)

ss = Normalizer(copy=False)

clf = ExtraTreesClassifier(
    #max_leaf_nodes=None, 
    min_samples_split=19,
    n_estimators=1000, 
    #max_depth=None, 
    max_features=0.6513343769448696, 
    min_samples_leaf=11, 
    #criterion='gini',
    class_weight='balanced'
)

pl = Pipeline([('ss', ss), ('pp', gus), ('clf', clf)])

param_grid = {
    #'pp__param':[0.05,  0.18475], 
             # 'clf__max_features':[ 0.65133, 0.8]
}

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=42)

model = GridSearchCV(
    pl, param_grid, 
    scoring='roc_auc',
    verbose=0, n_jobs = -1, cv = rskf, iid=False
)
model.fit(X_train, y_train).best_score_

In [ ]:
model.best_params_